# 📈 Facebook Ad Campaign Analysis

This notebook covers an end-to-end project using Facebook ad campaign data to:
- Explore key campaign metrics and customer segments (EDA)
- Simulate and evaluate A/B tests for ad effectiveness
- Predict ad click-through behavior using TensorFlow

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
# Load Data
df = pd.read_csv("facebook_ads.csv")
df.head()

## 🔍 Exploratory Data Analysis (EDA)

In [ ]:
df.info()
df.describe()

In [ ]:
# CTR distribution and campaign performance
df['CTR'] = df['Clicks'] / df['Impressions']
sns.histplot(df['CTR'], kde=True)
plt.title("Click-Through Rate Distribution")
plt.show()

In [ ]:
# CTR by gender
sns.barplot(data=df, x='Gender', y='CTR')
plt.title("CTR by Gender")
plt.show()

## 🧪 A/B Test Simulation

In [ ]:
np.random.seed(42)
df['ad_variant'] = np.random.choice(['A', 'B'], size=len(df))
grouped = df.groupby('ad_variant').agg({
    'Impressions': 'sum',
    'Clicks': 'sum'
})
grouped['CTR'] = grouped['Clicks'] / grouped['Impressions']
grouped

In [ ]:
clicks = grouped['Clicks'].values
impressions = grouped['Impressions'].values
stat, pval = proportions_ztest(clicks, impressions)
print(f"Z-statistic: {stat:.2f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("✅ Statistically significant difference in CTR between Ad A and B")
else:
    print("❌ No significant difference in CTR")

## 🤖 Predicting Clicks with TensorFlow

In [ ]:
# Prepare target
df['Clicked'] = (df['Clicks'] > 0).astype(int)

In [ ]:
# Encode categorical features
features = ['Gender', 'Age', 'ad_variant']
df_model = df[features + ['Clicked']].copy()

le_dict = {}
for col in features:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le

In [ ]:
# Train-test split
X = df_model[features].values
y = df_model['Clicked'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build DNN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
y_prob = model.predict(X_test)
print(f"AUC: {roc_auc_score(y_test, y_prob):.3f}")